In [ ]:
!pip install transformers torch accelerate datasets scikit-learn pandas numpy

In [ ]:
!pip install pandas groq tqdm --quiet
!pip install groq
import pandas as pd
from groq import Groq
from tqdm.notebook import tqdm
import time

# Load your dataset (Make sure you uploaded it to Colab)
file_path = "/content/symptoms_data_with_all_the_ct_values.csv"
df = pd.read_csv(file_path)
print(df.head())
# df_cleaned = df.dropna(subset=['student_answer'])
# print(df_cleaned)
# Use only the first 500 rows
df = df

# Initialize the Groq client
client = Groq(api_key="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")  # Replace with your actual key

# Prompt formatting pattern (for consistent LLM output)
# List of binary symptom columns
symptom_columns = [
    'fever', 'cough', 'breathlessness', 'body_ache',
    'vomiting', 'sore_throat', 'diarrhoea', 'sputum', 'nausea',
    'nasal_discharge', 'loss_of_taste', 'loss_of_smell', 'abdominal_pain',
    'chest_pain', 'haemoptsis', 'head_ache', 'body_pain', 'weak_ness',
    'cold'
]
# Define evaluation function
def get_present_symptoms(row):
    present = [symptom.replace("_", " ").capitalize() for symptom in symptom_columns if row[symptom] == 1]
    return present if present else ["None"]

def build_prompt(row):
    symptoms_list = get_present_symptoms(row)
    symptoms_text = ', '.join(symptoms_list)

    # Adjust based on your dataset column names
    text = f"The patient is a {row['age']} year old ."
    text += f" Symptoms include: {symptoms_text}."
    text += f" RT-PCR CT values are: Screening = {row['ct_value_screening']}, RdRp = {row['ct_value_rdrp']}, ORF1b = {row['ct_value_orf1b']}."

    if 'history' in row:
        text += f" Medical history: {row['underlying_medical_condition']}."
    return text.strip()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.8 MB/s eta 0:00:00
   age  underlying_medical_condition final_test_result  generated_result  \
0   60                             1          Positive                 1   
1   33                             1          Positive                 1   
2   17                             1          Positive                 1   
3   31                             1          Positive                 1   
4   27                             1          Positive                 1   

   ct_value_screening  ct_value_rdrp  ct_value_orf1b  fever  cough  \
0                24.0           27.0            24.0      1      1   
1                26.0           30.0            27.0      1      1   
2                19.0           23.0            20.0      1      0   
3                21.0           25.0            22.0      1      0   
4                22.0           27.0            24.0      1      0   

   breathlessness  ...  nasal_discharge  l

In [ ]:
# prompt: use the hagiung face api to call a llm and run a test wheather the LLm is giving out put or not

from transformers import pipeline

# Load a suitable model for text generation (choose a relatively small one for faster testing)
# You might need to experiment with different model names.
# Examples: 'distilgpt2', 'gpt2', 'EleutherAI/gpt-neo-125m'
model_name = "EleutherAI/gpt-neo-125m"

try:
    generator = pipeline("text-generation", model=model_name)
    print(f"Successfully loaded model: {model_name}")

    # Create a simple test prompt
    test_prompt = "Once upon a time,"

    # Generate a short response
    test_output = generator(test_prompt, max_length=50, num_return_sequences=1)

    if test_output and len(test_output) > 0 and 'generated_text' in test_output[0]:
        print("\nHugging Face API call successful. LLM is providing output.")
        print("Test Output:", test_output[0]['generated_text'])
    else:
        print("\nHugging Face API call successful, but LLM output is not in expected format.")
        print("Raw Output:", test_output)

except Exception as e:
    print(f"\nError during Hugging Face API call: {e}")
    print("Could not load the model or generate text. Please check the model name and your environment.")



config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Successfully loaded model: EleutherAI/gpt-neo-125m

Hugging Face API call successful. LLM is providing output.
Test Output: Once upon a time, the world was a place of the most beautiful and beautiful things.

The world was a place of the most beautiful and beautiful things.

The world was a place of the most beautiful and beautiful things.




In [ ]:
bins=[0, 20, 30,200]
bins
labels=['High_risk', 'Modarate_risk', 'Low_risk']
df['Risk_lebel'] = pd.cut(df['ct_value_rdrp'], bins, labels=labels)
df.info()
first_row = df.iloc[0]
first_row

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25634 entries, 0 to 25633
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   age                           25634 non-null  int64   
 1   underlying_medical_condition  25634 non-null  int64   
 2   final_test_result             25634 non-null  object  
 3   generated_result              25634 non-null  int64   
 4   ct_value_screening            25634 non-null  float64 
 5   ct_value_rdrp                 25634 non-null  float64 
 6   ct_value_orf1b                25634 non-null  float64 
 7   fever                         25634 non-null  int64   
 8   cough                         25634 non-null  int64   
 9   breathlessness                25634 non-null  int64   
 10  body_ache                     25634 non-null  int64   
 11  vomiting                      25634 non-null  int64   
 12  sore_throat                   25634 non-null  

,0
age,60
underlying_medical_condition,1
final_test_result,Positive
generated_result,1
ct_value_screening,24.0
ct_value_rdrp,27.0
ct_value_orf1b,24.0
fever,1
cough,1
breathlessness,1


In [ ]:
# Assuming df is already loaded and contains the data
# Assuming the `generator` pipeline is already initialized and working from the previous code

# Function to generate text based on a single row using the loaded model
def generate_text_from_row(row, generator, max_length=100):
    prompt = build_prompt(row) # Use the existing build_prompt function
    try:
        # Generate text using the Hugging Face pipeline
        generated_output = generator(prompt, max_length=max_length, num_return_sequences=1, do_sample=True)[0]['generated_text']
        return generated_output
    except Exception as e:
        print(f"Structure the data into text data: {e} also don't add extra information only take the row information and consider only the target variable as the final statment")
        return "Error generating text."

# Example: Generate text for the first row of the dataframe
if not df.empty:
    first_row = df.iloc[0]
    generated_story = generate_text_from_row(first_row, generator)
    print("\nGenerated Text for the first row:")
    print(generated_story)
else:
    print("DataFrame is empty. Cannot generate text.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Text for the first row:
The patient is a 60 year old . Symptoms include: Fever, Cough, Breathlessness. RT-PCR CT values are: Screening = 24.0, RdRp = 27.0, ORF1b = 24.0. Diagnosis = CT = 28.5 and RdRp = 29.0/cM = 8.1/cM = 8.5 b = (7.4/cM); FWE: Cough = 6.5


In [ ]:
import openai
from tqdm import tqdm

# openai.api_key = "sk-..."  # Replace with your OpenAI API Key
client = Groq(api_key="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")  # Replace with your actual key
from groq import Groq

client = Groq(api_key="your_groq_api_key")

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Convert the following tabular medical data into a natural language report:\n..."}
    ]
)
print(response.choices[0].message.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [ ]:
df['text_data']

NameError: name 'df' is not defined

In [ ]:
# Final label is 'final_test_result'
df[['text_data', 'final_test_result']].to_csv('final_llm_training_data.csv', index=False)